# Coursera Capstone Project - The Battle Of Neighborhoods

## Introduction
---------------------------------------
#### a. Background
Indonesian food is probably the tastiest invisible food in the world. It has been ranked highly by those who tried it, including recently by CNN, but many still do not know about it. It is not common to see Indonesian restaurant, much less than Thai or Japanese in many big cities around the world outside Indonesia. I suspect the same case is true for Toronto. Therefore, my hypothesis is that, if we develop the business of creating Indonesian restaurant there in the right way, it will become a successful business.

#### b. Problem
One of the biggest problems in creating restaurant is finding the right location. Where should we open our Indonesian restaurant? We might want to avoid area where there is already Indonesian restaurant (if any) or to some extent Chinese restaurant as it has slight similarity with Indonesian restaurant.

#### c. Interest
Hopefully this will bring benefit to anyone, especially entrepreneurs, who are interested in opening Indonesian restaurant business particularly in Toronto.


## Data Acquisition and Cleaning
---------------------------------------


#### Import necessary libraries and modules

In [1]:
import requests # library to handle requests
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
import urllib.request

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

!pip install beautifulsoup4
#!pip install lxml
from bs4 import BeautifulSoup

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Folium installed
Libraries imported.


#### Get the first source of data - list of neighborhoods
Get the data from Wikipedia URL and parse using BeautifulSoup

In [2]:
fp = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(fp)
   
table = soup.find('table', attrs={'wikitable sortable'})
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighborhood"])
df.head(10) #check the first 10 data

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


Delete rows whose Borough is not assigned

In [3]:
# Get names of indexes for which column Borough is not assigned
indexNames = df[ df['Borough'] == 'Not assigned'].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.head(10)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


Of the remaining rows, set Neighborhood = Borough if it is not assigned

In [4]:
# Get names of indexes for which column Neighborhood is not assigned
indexNames = df[ df['Neighborhood'] == 'Not assigned'].index
df.loc[indexNames,'Neighborhood'] = df.loc[indexNames,'Borough']
df.head(10)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


Group the neighborhoods having the same postal code

In [5]:
new_df = df.groupby(['Postcode', 'Borough']).agg({'Neighborhood':lambda x:', '.join(x)}).reset_index()
new_df.columns = ['PostalCode', 'Borough', 'Neighborhood']
new_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### Get the second source of data - obtaining latitude and longitude of the neighborhoods
Get the data from Geocoder package

In [6]:
df_latlong = pd.read_csv('https://cocl.us/Geospatial_data')
df_latlong.columns = ['PostalCode','Latitude','Longitude']
df_latlong

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Append the list of neighborhood data and its latitude and longitude

In [7]:
neighborhoods = pd.merge(new_df, df_latlong, on='PostalCode')
neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


#### Define Foursquare Credentials and Version

In [8]:
CLIENT_ID = 'F2RGG0JZUAGWR53ILSX23G1ZNMQFTSSRS04WD5TR1RVOJSKS' # your Foursquare ID
CLIENT_SECRET = '5PGTYZ12HRL14AM4HAVC2FAFAK2WZTW0Z2IXSDTXI13ZFDY4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F2RGG0JZUAGWR53ILSX23G1ZNMQFTSSRS04WD5TR1RVOJSKS
CLIENT_SECRET:5PGTYZ12HRL14AM4HAVC2FAFAK2WZTW0Z2IXSDTXI13ZFDY4


We know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Methodology
-------------------------------
#### Explore Venues in Toronto
Using Foursquare API, get nearby venues in each of the neighborhoods in Toronto.

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [13]:
radius=500
LIMIT=100

toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

Let's check the size of the resulting dataframe

In [14]:
print(toronto_venues.shape)
toronto_venues.head()

(2236, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


#### Look for “Indonesian restaurant”
Let's find out how many of the venues are Indonesian Restaurant.

In [15]:
# Get names of indexes for which Category is Indonesian Restaurant
indexNames = toronto_venues[ toronto_venues['Venue Category'] == 'Indonesian Restaurant'].index
indexNames

Int64Index([174], dtype='int64')

#### Look for “Chinese restaurant”
Apparently there is only 1 Indonesian Restaurant in Toronto! That makes it easy - we can just develop Indonesian Restaurant in any place except one with that restaurant. However, let us broaden the search to also include Chinese Restaurant as although not that similar, Indonesian food has slight similarity with Chinese food.

In [16]:
# Get names of indexes for which Category is Chinese Restaurant
indexNames = toronto_venues[ toronto_venues['Venue Category'] == 'Chinese Restaurant'].index
indexNames

Int64Index([  48,   63,   65,   79,   80,  153,  163,  237,  263,  486,  601,
             669,  815, 1021, 1022, 1208, 1555, 1607, 1619, 1620, 1636, 2115,
            2215],
           dtype='int64')

It seems more promising. We can then find which neighborhoods contain a lot of Indonesian or Chinese Restaurant, which contain few, and which contain none. Let us reshape the data to include each of the venues category as column.

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category and specifically choose "Indonesian Restaurant" and "Chinese Restaurant" columns.

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped = toronto_grouped[['Neighborhood','Chinese Restaurant','Indonesian Restaurant']]
toronto_grouped

,Neighborhood,Chinese Restaurant,Indonesian Restaurant
0,"Adelaide, King, Richmond",0.00,0.0
1,Agincourt,0.25,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.0
4,"Alderwood, Long Branch",0.00,0.0
...,...,...,...
95,Willowdale West,0.00,0.0
96,Woburn,0.00,0.0
97,"Woodbine Gardens, Parkview Hill",0.00,0.0
98,Woodbine Heights,0.00,0.0


Let's print each neighborhood along with frequency of both Indonesian Restaurant and Chinese Restaurant.

In [21]:
num_top_venues = 2

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                   venue  freq
0     Chinese Restaurant   0.0
1  Indonesian Restaurant   0.0


----Agincourt----
                   venue  freq
0     Chinese Restaurant  0.25
1  Indonesian Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                   venue  freq
0     Chinese Restaurant   0.0
1  Indonesian Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                   venue  freq
0     Chinese Restaurant   0.0
1  Indonesian Restaurant   0.0


----Alderwood, Long Branch----
                   venue  freq
0     Chinese Restaurant   0.0
1  Indonesian Restaurant   0.0


----Bathurst Manor, Downsview North, Wilson Heights----
                   venue  freq
0     Chinese Restaurant  0.05
1  Indonesian Restaurant  0.00


----Bayview Village----
                   venue  freq
0     Chinese Restaurant  0.25
1  Indonesian Restauran

#### Use K-Means to Cluster Neighborhoods

In [22]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster.

In [24]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_grouped

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods.set_index('Neighborhood'), on='Neighborhood')

# Get names of indexes for which column Cluster is not assigned
indexNames = toronto_merged[pd.isnull(toronto_merged['Cluster Labels'])].index
# Delete these row indexes from dataFrame
toronto_merged.drop(indexNames , inplace=True)

toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].apply(np.int64)

toronto_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,Chinese Restaurant,Indonesian Restaurant,PostalCode,Borough,Latitude,Longitude
0,0,"Adelaide, King, Richmond",0.00,0.0,M5H,Downtown Toronto,43.650571,-79.384568
1,1,Agincourt,0.25,0.0,M1S,Scarborough,43.794200,-79.262029
2,0,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.0,M1V,Scarborough,43.815252,-79.284577
3,0,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.0,M9V,Etobicoke,43.739416,-79.588437
4,0,"Alderwood, Long Branch",0.00,0.0,M8W,Etobicoke,43.602414,-79.543484


Finally, let's visualize the resulting clusters

In [26]:
# create map

address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
toronto_merged['Cluster Labels']
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #print("rainbow cluster: " + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Result 
----------------------------------------
We will examine each of the clusters.

#### Cluster 0
This is cluster where there is no or few Indonesian or Chinese restaurants.

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Cluster Labels,Neighborhood,Chinese Restaurant,Indonesian Restaurant,PostalCode,Borough,Latitude,Longitude
0,0,"Adelaide, King, Richmond",0.0,0.0,M5H,Downtown Toronto,43.650571,-79.384568
2,0,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,M1V,Scarborough,43.815252,-79.284577
3,0,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,M9V,Etobicoke,43.739416,-79.588437
4,0,"Alderwood, Long Branch",0.0,0.0,M8W,Etobicoke,43.602414,-79.543484
7,0,"Bedford Park, Lawrence Manor East",0.0,0.0,M5M,North York,43.733283,-79.419750
...,...,...,...,...,...,...,...,...
95,0,Willowdale West,0.0,0.0,M2R,North York,43.782736,-79.442259
96,0,Woburn,0.0,0.0,M1G,Scarborough,43.770992,-79.216917
97,0,"Woodbine Gardens, Parkview Hill",0.0,0.0,M4B,East York,43.706397,-79.309937
98,0,Woodbine Heights,0.0,0.0,M4C,East York,43.695344,-79.318389


#### Cluster 1
This is cluster where there are some Indonesian or Chinese restaurants.

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Cluster Labels,Neighborhood,Chinese Restaurant,Indonesian Restaurant,PostalCode,Borough,Latitude,Longitude
1,1,Agincourt,0.250000,0.0,M1S,Scarborough,43.794200,-79.262029
6,1,Bayview Village,0.250000,0.0,M2K,North York,43.786947,-79.385975
34,1,"Dorset Park, Scarborough Town Centre, Wexford ...",0.166667,0.0,M1P,Scarborough,43.757410,-79.273304
61,1,L'Amoreaux West,0.125000,0.0,M1W,Scarborough,43.799525,-79.318389
92,1,Westmount,0.166667,0.0,M9P,Etobicoke,43.696319,-79.532242


#### Cluster 2
This is cluster where there are many Indonesian or Chinese restaurants.

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Cluster Labels,Neighborhood,Chinese Restaurant,Indonesian Restaurant,PostalCode,Borough,Latitude,Longitude
5,2,"Bathurst Manor, Downsview North, Wilson Heights",0.047619,0.0,M3H,North York,43.754328,-79.442259
19,2,Central Bay Street,0.024390,0.0,M5G,Downtown Toronto,43.657952,-79.387383
20,2,"Chinatown, Grange Park, Kensington Market",0.047619,0.0,M5T,Downtown Toronto,43.653206,-79.400049
24,2,"Clarks Corners, Sullivan, Tam O'Shanter",0.076923,0.0,M1T,Scarborough,43.781638,-79.304302
45,2,"Flemingdon Park, Don Mills South",0.045455,0.0,M3C,North York,43.725900,-79.340923
49,2,"Harbord, University of Toronto",0.027778,0.0,M5S,Downtown Toronto,43.662696,-79.400049
68,2,North Toronto West,0.043478,0.0,M4R,Central Toronto,43.715383,-79.405678
73,2,Queen's Park,0.026316,0.0,M7A,Downtown Toronto,43.662301,-79.389494
73,2,Queen's Park,0.026316,0.0,M9A,Queen's Park,43.667856,-79.532242


## Conclusion
----------------------------------
Out of around 100 neighborhoods, only 9 neighborhoods have many Indonesian or Chinese restaurants, specifically those belong in Cluster 2, for example, Queen's Park. More than 80% of the neighborhoods contain no or very few Indonesian or Chinese restaurants, specifically those belong in Cluster 0. I recommend those who want to open Indonesian restaurant to start by looking areas in neighborhoods in Cluster 0.